In [ ]:
%load_ext watermark


In [ ]:
from Bio import Phylo as BioPhylo
import alifedata_phyloinformatics_convert as apc
import covasim as cv
from hstrat import _auxiliary_lib as hstrat_aux
from matplotlib import patches as mpl_patches
from matplotlib import pyplot as plt
import numpy as np
from teeplot import teeplot as tp

from pylib._cv_infection_log_to_alstd_df import cv_infection_log_to_alstd_df


In [ ]:
%watermark -diwmuv -iv


In [ ]:
teeplot_subdir = "2025-03-31-dev-variantsync"
teeplot_subdir


## Setup Params

Define `gamma` and `delta` strains, as well as accompanying lower-transmissibility `gamma'` and `delta'` strains.


In [ ]:
pop_size = 10_000
num_variants = 5  # 0th variant is wild-type


In [ ]:
# from https://github.com/InstituteforDiseaseModeling/covasim/blob/fef033fed5a0ca71be9c4f773d75dce1d237e753/covasim/parameters.py
gamma = dict(
    rel_beta=2.05,  # Estimated to be 1.7-2.4-fold more transmissible than wild-type: https://science.sciencemag.org/content/early/2021/04/13/science.abh2644
    rel_symp_prob=1.0,
    rel_severe_prob=2.6,  # From https://www.eurosurveillance.org/content/10.2807/1560-7917.ES.2021.26.16.2100348
    rel_crit_prob=1.0,
    rel_death_prob=1.0,
)
gamma_ = {**gamma, "rel_beta": 1.5}  # lower-transmissibility variant

delta = dict(
    rel_beta=2.2,  # Estimated to be 1.25-1.6-fold more transmissible than B117: https://www.researchsquare.com/article/rs-637724/v1
    rel_symp_prob=1.0,
    rel_severe_prob=3.2,  # 2x more transmissible than alpha from https://mobile.twitter.com/dgurdasani1/status/1403293582279294983
    rel_crit_prob=1.0,
    rel_death_prob=1.0,
)
delta_ = {**delta, "rel_beta": 1.6}  # lower-transmissibility variant


In [ ]:
# import gamma and then delta 30 days later
gamma = cv.variant(variant=gamma, label="gamma", days=0, n_imports=10)
delta = cv.variant(variant=delta, label="delta", days=30, n_imports=100)

# lower-transimissibility variants arise from spontaneous mutations, not imports
gamma_ = cv.variant(variant=gamma_, label="gamma'", days=0, n_imports=0)
delta_ = cv.variant(variant=delta_, label="delta'", days=0, n_imports=0)


## Prototype Within-host Dynamics Extension


In [ ]:
class SyncWithinHost:

    _host_compartments: np.ndarray

    def __init__(
        self: "SyncWithinHost", pop_size: int, num_variants: int
    ) -> None:
        shape = (pop_size, num_variants)
        self._host_compartments = np.zeros(shape, dtype=float)

    def __call__(self: "SyncWithinHost", sim: cv.Sim) -> None:
        compartments = self._host_compartments
        people = sim.people
        random_p = np.random.rand(*people["infectious_variant"].shape)

        ## sync covasim to host compartments
        #######################################################################
        # zero out non-infectious/exposed compartments
        mask = ~(people["infectious"] | people["exposed"])
        compartments[mask, :] = 0.0

        # ensure host compartments are initialized w/ covasim infectious variant
        for variant in range(1, num_variants):
            compartments[:, variant] = np.maximum(
                people["infectious_variant"] == variant,
                compartments[:, variant],
            )

        # update host compartments
        #######################################################################
        # grow strains
        compartments[:, 1] *= 2.0
        compartments[:, 2] *= 3.0  # lower-trans variant, faster growth
        compartments[:, 3] *= 2.0
        compartments[:, 4] *= 3.0  # lower-trans variant, faster growth

        # introduce low-transmissibility variants thru spontaneous mutation
        # of high-transmissibility variants
        # e.g., gamma -> gamma' and delta -> delta'
        for lowtran_variant in 2, 4:
            p = 1.0 - np.power(0.95, compartments[:, lowtran_variant - 1])
            compartments[:, lowtran_variant] = np.maximum(
                random_p < p,
                compartments[:, lowtran_variant],
            )

        ## sync host compartments to covasim "infectious variant"
        #######################################################################
        # sample current infectious variant from compartments
        compartments_ = compartments.copy()
        compartments_ *= np.random.rand(*compartments.shape)
        compartments_[1, :] *= 1.5  # higher-transmissiblity bonus
        compartments_[3, :] *= 1.5  # higher-transmissiblity bonus
        sampled_strains = np.where(
            compartments_.any(axis=1),
            np.argmax(compartments_, axis=1),
            np.nan,
        )

        # update current covasim infectious variant
        people["infectious_variant"] = np.where(
            ~np.isnan(people["infectious_variant"]),
            sampled_strains,
            people["infectious_variant"],
        )


## Run Example Simulation


In [ ]:
sim = cv.Sim(
    interventions=[SyncWithinHost(pop_size, num_variants)],
    pop_infected=0,  # disable wild-type strain
    pop_size=pop_size,
    variants=[gamma, gamma_, delta, delta_],
    rand_seed=1,
)
sim.run()


In [ ]:
tp.tee(
    sim.plot,
    "variant",
    teeplot_outattrs={"what": "within-host"},
    teeplot_subdir=teeplot_subdir,
)
...


## Visualize Pathogen Phylogeny


In [ ]:
phylo_df = cv_infection_log_to_alstd_df(sim.people.infection_log)

phylo_df = hstrat_aux.alifestd_downsample_tips_asexual(
    phylo_df, n_downsample=200
)

phylo_tree = apc.alife_dataframe_to_biopython_tree(
    phylo_df,
    setattrs=["variant"],
    setup_branch_lengths=True,
)

node_colors = {
    "delta": "blue",
    "delta'": "green",
    "gamma": "red",
    "gamma'": "purple",
}
for node in phylo_tree.find_clades():
    node.color = node_colors[node.variant]


In [ ]:
def draw_biophylo(tree: BioPhylo.BaseTree, **kwargs) -> plt.Axes:
    ax = plt.gca()
    BioPhylo.draw(tree, axes=ax, do_show=False, **kwargs)
    return ax


with tp.teed(
    draw_biophylo,
    phylo_tree,
    teeplot_outattrs={"what": "within-host"},
    teeplot_outdir=teeplot_subdir,
) as ax:
    ax.set_xlabel("Days")
    ax.yaxis.set_visible(False)
    ax.spines["left"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)

    legend_handles = [
        mpl_patches.Patch(color=color, label=variant)
        for variant, color in node_colors.items()
    ]
    ax.legend(handles=legend_handles, loc="upper right")
